In [1]:
import numpy as np
import pandas as pd
import scipy.stats as sp
import pywt
import pickle

#INPUT
Number_of_Data=100
Number_of_Sensor=2
Number_of_Feature=9

#Mother Wavelet 및 level 설정
wavelet=pywt.Wavelet('sym2')
Level=12
select=9

class MakingFeature():
    def __init__(self,wavelet,Level,select):
        self.wavelet=pywt.Wavelet('sym2')
        self.Level=Level
        self.select=select
    def rms(self,y): #rms 함수 정의
        return np.sqrt(np.mean(y**2))
    #Time Domain Feature 추출
    def Transform(self,x):
        TD_Normal = np.zeros(shape=(Number_of_Sensor*Number_of_Feature))
        path2 = x
        data2 = pd.read_csv(path2)
        a=np.array(data2,dtype=np.float32)
        for j in np.arange(0,Number_of_Sensor):
            TD_Normal[Number_of_Feature*j+0]=np.max(a[:,j])
            TD_Normal[Number_of_Feature*j+1]=np.min(a[:,j])
            TD_Normal[Number_of_Feature*j+2]=np.mean(a[:,j])
            TD_Normal[Number_of_Feature*j+3]=np.var(a[:,j])
            TD_Normal[Number_of_Feature*j+4]=self.rms(a[:,j])
            TD_Normal[Number_of_Feature*j+5]=sp.skew(a[:,j])
            TD_Normal[Number_of_Feature*j+6]=sp.kurtosis(a[:,j])
            TD_Normal[Number_of_Feature*j+7]=self.rms(a[:,j])/np.mean(a[:,j])
            TD_Normal[Number_of_Feature*j+8]=np.max(a[:,j])/np.mean(a[:,j])
            #Normal[10*j+9,j]=rms(a[:,j])    
    #Frequency Domain Feature 추출
        FD_Normal = np.zeros(shape=(Number_of_Sensor*Number_of_Feature*select))
        path2 = x
        data2 = pd.read_csv(path2)
        b=np.array(data2,dtype=np.float32)
        Coef2=pywt.wavedec(b,wavelet,level=Level,axis=0)#Wavelet Transform Coefficient
        for j in np.arange(Number_of_Sensor):
            for k in np.arange(select):
                coef2=Coef2[Level-k]
                FD_Normal[Number_of_Feature*j*select+k*Number_of_Feature+0]=np.max(coef2[:,j])
                FD_Normal[Number_of_Feature*j*select+k*Number_of_Feature+1]=np.min(coef2[:,j])
                FD_Normal[Number_of_Feature*j*select+k*Number_of_Feature+2]=np.mean(coef2[:,j])
                FD_Normal[Number_of_Feature*j*select+k*Number_of_Feature+3]=np.var(coef2[:,j])
                FD_Normal[Number_of_Feature*j*select+k*Number_of_Feature+4]=self.rms(coef2[:,j])
                FD_Normal[Number_of_Feature*j*select+k*Number_of_Feature+5]=sp.skew(coef2[:,j])
                FD_Normal[Number_of_Feature*j*select+k*Number_of_Feature+6]=sp.kurtosis(coef2[:,j])
                FD_Normal[Number_of_Feature*j*select+k*Number_of_Feature+7]=self.rms(coef2[:,j])/np.mean(coef2[:,j])
                FD_Normal[Number_of_Feature*j*select+k*Number_of_Feature+8]=np.max(coef2[:,j])/np.mean(coef2[:,j])
    
        FT=np.concatenate((TD_Normal,FD_Normal),axis=0)
    #Time Domain과 Frequency Domain 특성값 합치기
    
    #p-value 기반 상위 특징요소 선택하기
        with open('Select.pickle','rb') as f:
            b=pickle.load(f)
        Rank= 10
        New_Feature=np.zeros(shape=(Rank))
        for i in np.arange(Rank):
        
            New_Feature[i]=np.array(FT[int(b[i,1])])
        return New_Feature

a=MakingFeature()
q=a.Transform('./Normal/Normal_1.csv')